# Build Pipeline with Components from yaml

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define and load `commandComponent` from YAML
- Create `pipeline` using loaded component.

**Motivations** - This notebook covers the scenario that user define components using yaml then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput, load_component
from azure.ml._constants import AssetTypes

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [ ]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    # Exclude 
    # credential = DefaultAzureCredential(exclude_environment_credential=True)
    # ...
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
# check if there are workspace config in azureml-examples/sdk/jobs folder
config_path = "../../.azureml/config.json"
try:
    ml_client = MLClient.from_config(credential=credential, path=config_path)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    subscription_id = "<your subscription ID>"
    resource_group = "<your workspace name>"
    workspace_name = "<your resource group>"

    if subscription_id.startswith('<'):
        raise ex
    else:  # write and reload from config file
        config = {
            "Scope": "/subscriptions/"
            + subscription_id
            + "/resourceGroups/"
            + resource_group
            + "/providers/Microsoft.MachineLearningServices/workspaces/"
            + workspace_name
            + "/projects/samples"
        }
        import json
        import os

        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

## 1.4. Retrieve or create an Azure Machine Learning compute target

In [ ]:
from azure.ml.entities import AmlCompute

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    compute = AmlCompute(
        name=cluster_name,
        size="Standard_D2_v2",
        max_instance=2
    )
    ml_client.compute.begin_create_or_update()

# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [ ]:
parent_dir = '.'
train_model = load_component(yaml_file=parent_dir + "/train_model.yml")
score_data = load_component(yaml_file=parent_dir + "/score_data.yml")
eval_model = load_component(yaml_file=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [ ]:
# print the component as yaml
print(train_model)

In [ ]:
# inspect more information
print(type(train_model))
help(train_model._func)

# 3. Sample pipeline job
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@dsl.pipeline(
    default_compute="cpu-cluster",
    description="E2E dummy train-score-eval pipeline with components defined via yaml",
)
def pipeline_with_components_from_yaml(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, 
        test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }

pipeline = pipeline_with_components_from_yaml(
    training_input = JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    test_input = JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based"
)

In [ ]:
print(pipeline)

## 3.2 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="pipeline_samples")
print(f'Job link: {pipeline_job.services["Studio"].endpoint}')

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)